In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_cohere import CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever


from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser


import os
from dotenv import dotenv_values


In [11]:
config = dotenv_values(dotenv_path='../.env')

In [12]:
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']
os.environ['COHERE_API_KEY'] = config['COHERE_API_KEY']

# Definição de LLMs

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4, max_tokens=500)

# Carregar PDF e instanciar

In [5]:
path_file = './os-sertoes.pdf'

loader = PyPDFLoader(path_file, extract_images=False)

pages = loader.load_and_split()

# Definição de splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [7]:
chunks = text_splitter.split_documents(pages)

In [8]:
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="rerank_rag_db"
)

In [9]:
naive_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 10}
)

In [14]:
rerank = CohereRerank(
  top_n=5,
  model="rerank-english-v2.0"
)

In [18]:
compressor_rerank_retrievel = ContextualCompressionRetriever(
  base_compressor=rerank,
  base_retriever=naive_retriever
)

In [19]:
template_quests = '''
Você é um especialista no livro Os sertões de Euclides da Cunha. Deve responder objetivamente a questão do usuario com base no conteudo fornecido de contexto

Questão do usuario: {query}


Conteudo: {content}
'''

rag_prompt = ChatPromptTemplate.from_template(template_quests)

In [20]:
setup_retrievel = RunnableParallel({
    "query": RunnablePassthrough(),
    "content": compressor_rerank_retrievel,
})

In [21]:
output_parser = StrOutputParser()

In [23]:
rerank_chain = setup_retrievel | rag_prompt | llm | output_parser

# Perguntas

In [24]:
rerank_chain.invoke('Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?')

'Euclides da Cunha descreve o ambiente natural do sertão nordestino como árido, hostil e desafiador. Ele destaca a seca, a aridez do solo, as secas prolongadas e as condições climáticas extremas como elementos marcantes desse ambiente. Essas características influenciam diretamente a vida dos habitantes do sertão, tornando a sobrevivência uma luta constante. A escassez de água, a falta de recursos naturais e as condições climáticas adversas impactam a agricultura, a pecuária e a economia da região, contribuindo para a pobreza e a desigualdade social. Euclides da Cunha também destaca a resistência e a força do povo sertanejo diante dessas adversidades, ressaltando sua capacidade de adaptação e sua relação íntima com a natureza árida e implacável do sertão.'

In [25]:
rerank_chain.invoke('Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?')

'Euclides da Cunha descreve a população sertaneja como sendo marcada pela resistência física, pela bravura, pela adaptabilidade às condições adversas do ambiente e pela fé religiosa intensa. Ele relaciona essas características com o ambiente em que vivem ao destacar a influência do clima árido e das condições geográficas do sertão, que moldam a personalidade e o modo de vida dessas pessoas. Além disso, ele ressalta a relação dos sertanejos com a natureza, destacando sua luta constante contra as secas e as adversidades do meio ambiente.'

In [26]:
rerank_chain.invoke('Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?')

'O contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha em Os sertões, foi marcado pela situação de extrema pobreza e marginalização social dos sertanejos do nordeste brasileiro, especialmente na região de Canudos, na Bahia. A população local sofria com a seca, a fome, a falta de assistência do governo e a opressão dos latifundiários. Além disso, havia um sentimento de revolta e resistência contra a República recém-instaurada no Brasil, que era vista como distante e indiferente às necessidades do povo do sertão. A liderança carismática de Antônio Conselheiro e a formação de uma comunidade religiosa e autossuficiente em Canudos contribuíram para a polarização e o conflito com as autoridades republicanas, que culminou na Guerra de Canudos em 1896.'

In [27]:
rerank_chain.invoke('Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?')

'Euclides da Cunha descreve Antônio Conselheiro como um líder carismático e messiânico, capaz de atrair seguidores e criar uma comunidade religiosa em Canudos. Ele é retratado como um homem visionário, que pregava a volta às tradições e valores do sertão, desafiando a autoridade do governo e da Igreja. Seu papel na Guerra de Canudos é visto como fundamental, pois ele foi o líder espiritual e político dos sertanejos que resistiram às tropas do governo, tornando-se um símbolo de resistência e luta contra a opressão. Euclides da Cunha destaca a influência de Antônio Conselheiro na formação da comunidade de Canudos e na mobilização dos sertanejos, ressaltando sua importância histórica no conflito.'

In [28]:
rerank_chain.invoke('Quais são os principais aspectos da crítica social e política presentes em "Os Sertões"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?')

'Em "Os Sertões", Euclides da Cunha aborda principalmente a questão da formação do povo brasileiro, a luta entre civilização e barbárie, e a relação entre o homem e o meio ambiente no sertão nordestino. O autor realiza uma crítica social e política ao mostrar as condições precárias de vida dos sertanejos, a exploração dos trabalhadores rurais, a violência e a desigualdade social presentes na região.\n\nEsses aspectos refletem a visão do autor sobre o Brasil da época ao evidenciar as contradições e conflitos existentes no país, especialmente entre o progresso urbano e a realidade rural. Euclides da Cunha denuncia a violência e a injustiça social, além de destacar a resistência e a força do povo sertanejo diante das adversidades. Sua obra também critica a atuação do Estado e a falta de políticas públicas voltadas para o desenvolvimento e o bem-estar da população do sertão, evidenciando a necessidade de uma transformação social e política no Brasil.'